In [96]:
import csv
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
#import scipy as sp
#import math
from time import sleep
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.max_columns', None)

In [186]:
%matplotlib inline

In [95]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 10 14:12:11 2018

@author: rajat13440
"""
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from socket import timeout
import http
import gc
import nltk
import unicodedata
import re

summaryDict={}

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/words')
except LookupError:
     nltk.download('words')

#try:
#    nltk.data.find('sentiment/vader_lexicon')
#except LookupError:
#    nltk.download('vader_lexicon')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

#try:
#    nltk.data.find('tokenizers/punkt')
#except LookupError:
#    nltk.download('wordnet')

#class ProcessData:
#    
#    def __init__(self,x):
#        
#        self.GetSentenceTokens(x)
#        self.Normalize(x)

class Extract:
    def GetText(self,link):
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}
        try:
            req = Request(link, headers=headers)
            with urlopen(req) as response:
                page = response.read()
                soup=BeautifulSoup(page, 'html.parser')
                ptag=soup.find_all('p')
                ''' Analysis
                 #if len(ptag)<5:
                  #print("VERY LOW P TAGS, STUDY THIS - %s - %d" %(link, len(ptag)))
                  #texta=link + "  **VERY LOW P TAGS, STUDY THIS " '''
            text=[]
            for tag in ptag:
                text.append(tag.get_text())
                texta=str(' '.join(text)).replace('\n',' ')

            #print("processed {} {} \n".format(link,b))
            #b+=1
        except http.client.IncompleteRead:
            #print("INCOMPLETE READ - {}".format(link))
            texta="ERROR: "+link + "  **INCOMPLETE READ"
        except WindowsError as e:
            #print("WINDOWS ERROR {} {}".format(e,link))
            texta="ERROR: "+link + "  **WINDOWS ERROR {}".format(e)
        except BaseException as error:
            #print("SOME OTHER ERROR {}".format(link))
            texta="ERROR: "+link + "  **SOME OTHER ERROR  {}".format(error)
        except timeout:
            #print("LINK {} TIMED OUT \n".format(link))
            #timedout.append(link)
            texta="ERROR: "+link + "  **TIMED OUT"
        del text,headers
        gc.collect    
        return texta
    
    def _Frequency(self,wordslist):
        trimmeddictlist=[]
        freq={}
        for word in wordslist:
            freq[word]=wordslist.count(word)
            # gets only those words which are frequent than mean word frequency of that text
            trimmedfreq={k:v for (k,v) in freq.items() if v >(float(sum(freq.values())) / len(freq))}    
            trimmeddictlist.append(trimmedfreq)
        return trimmedfreq

    def Summary(self,article):
        #article=GetText(url)
        percent=(20/100)
        sentokens=self.GetSentenceTokens(article)
        tokens=self.Normalize(article)
        freqlist=self._Frequency(tokens)
        #sentencerank=[]
        sentf={}
        for sentence in sentokens:
            msentence=sentence.lower().split(' ')
            a=0
            for word in freqlist:
                if word.lower() in msentence:
                    a+=1
                    sentf[sentence]=a
        slength=percent*len(sentokens)
        sorteddict=sorted(sentf, key=sentf.get, reverse=True)[:int(slength)]
        #sentencerank.append(sorteddict)
    
        summary=str(' '.join(sorteddict))
        return summary
    

        
    def GetSentenceTokens(self,x):
        y=nltk.sent_tokenize(x)
        return y    
        
    def _remove_stopwords(self,words):
        """Remove stop words from list of tokenized words"""
        new_words = []
        for word in words:
            if word not in nltk.corpus.stopwords.words('english'):
                new_words.append(word)
        return new_words
    
    def _lower(self,x):
        norm=[word.lower() for word in x]
        return norm

    def _remove_non_ascii(self,words):
        """Remove non-ASCII characters from list of tokenized words"""
        new_words = []
        for word in words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return new_words

    def _remove_punctuation(self,words):
        """Remove punctuation from list of tokenized words"""
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        return new_words

    def Normalize(self,x):
        x1=nltk.word_tokenize(x)
        x2=self._remove_stopwords(x1)
        x3=self._remove_punctuation(x2)
        x4=self._remove_non_ascii(x3)
        x5=self._lower(x4)
        return x5
    
    def Summarize(self,url):
        import pandas as pd
        import datetime
        #import csv
        import numpy as np
        import time
        starttime=time.time()
        exclude=['twitter','glassdoor','youtube','wordpress','facebook','wikipedia','play.google','nasdaq']
        startTime_Main = datetime.datetime.now().strftime('%H:%M:%S')
        summarylist=[]
        summary1=""
        #print("---START TIME %s ---" % startTime_Main)
        if any (s in url for s in exclude):
            #print("Not allowed -",url)
            summary1="Not Allowed {}".format(url)
        else:
            try:
                article=self.GetText(url)
            except BaseException as e:
                article="ERROR {} {}".format(e,url)
            
            if not article.startswith("ERROR"):
                print("fetching summary for - ",url)
                summary1=self.Summary(article)
            else:
                summary1=article
        
        summaryDict[url]=summary1
        #print("{:.2f}".format((time.time()-starttime)))
        return summary1
        

In [157]:
# -*- coding: utf-8 -*-
"""
Created on Mon Sep 10 14:12:11 2018

@author: rajat13440
"""
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from socket import timeout
import http
import gc
import nltk
import unicodedata
import re
from sklearn.base import TransformerMixin,BaseEstimator
summaryDict={}

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/words')
except LookupError:
     nltk.download('words')

#try:
#    nltk.data.find('sentiment/vader_lexicon')
#except LookupError:
#    nltk.download('vader_lexicon')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

#try:
#    nltk.data.find('tokenizers/punkt')
#except LookupError:
#    nltk.download('wordnet')

#class ProcessData:
#    
#    def __init__(self,x):
#        
#        self.GetSentenceTokens(x)
#        self.Normalize(x)

class ExtractNew(BaseEstimator,TransformerMixin):
    def transform(self,links,**kwargs):
        textdict={}
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}
        for link in links:
            text=[]
            try:
                req = Request(link, headers=headers)
                with urlopen(req) as response:
                    page = response.read()
                    soup=BeautifulSoup(page, 'html.parser')
                    ptag=soup.find_all('p')
                    ''' Analysis
                     #if len(ptag)<5:
                      #print("VERY LOW P TAGS, STUDY THIS - %s - %d" %(link, len(ptag)))
                      #texta=link + "  **VERY LOW P TAGS, STUDY THIS " '''
                
                for tag in ptag:
                    text.append(tag.get_text())
                    texta=str(' '.join(text)).replace('\n',' ')
                
                #print("processed {} {} \n".format(link,b))
                #b+=1
            except http.client.IncompleteRead:
                #print("INCOMPLETE READ - {}".format(link))
                texta="ERROR: "+link + "  **INCOMPLETE READ"
            except WindowsError as e:
                #print("WINDOWS ERROR {} {}".format(e,link))
                texta="ERROR: "+link + "  **WINDOWS ERROR {}".format(e)
            except BaseException as error:
                #print("SOME OTHER ERROR {}".format(link))
                texta="ERROR: "+link + "  **SOME OTHER ERROR  {}".format(error)
            except timeout:
                #print("LINK {} TIMED OUT \n".format(link))
                #timedout.append(link)
                texta="ERROR: "+link + "  **TIMED OUT"
            textdict[link]=texta
            #del text,headers
            gc.collect    
        return textdict
    
    def fit(self,X, y=None):
        return self
    
    def _Frequency(self,wordslist):
        trimmeddictlist=[]
        freq={}
        for word in wordslist:
            freq[word]=wordslist.count(word)
            # gets only those words which are frequent than mean word frequency of that text
            trimmedfreq={k:v for (k,v) in freq.items() if v >(float(sum(freq.values())) / len(freq))}    
            trimmeddictlist.append(trimmedfreq)
        return trimmedfreq

    def Summary(self,article):
        #article=GetText(url)
        percent=(20/100)
        sentokens=self.GetSentenceTokens(article)
        tokens=self.Normalize(article)
        freqlist=self._Frequency(tokens)
        #sentencerank=[]
        sentf={}
        for sentence in sentokens:
            msentence=sentence.lower().split(' ')
            a=0
            for word in freqlist:
                if word.lower() in msentence:
                    a+=1
                    sentf[sentence]=a
        slength=percent*len(sentokens)
        sorteddict=sorted(sentf, key=sentf.get, reverse=True)[:int(slength)]
        #sentencerank.append(sorteddict)
    
        summary=str(' '.join(sorteddict))
        return summary
    

        
    def GetSentenceTokens(self,x):
        y=nltk.sent_tokenize(x)
        return y    
        
    def _remove_stopwords(self,words):
        """Remove stop words from list of tokenized words"""
        new_words = []
        for word in words:
            if word not in nltk.corpus.stopwords.words('english'):
                new_words.append(word)
        return new_words
    
    def _lower(self,x):
        norm=[word.lower() for word in x]
        return norm

    def _remove_non_ascii(self,words):
        """Remove non-ASCII characters from list of tokenized words"""
        new_words = []
        for word in words:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
        return new_words

    def _remove_punctuation(self,words):
        """Remove punctuation from list of tokenized words"""
        new_words = []
        for word in words:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        return new_words

    def Normalize(self,x):
        x1=nltk.word_tokenize(x)
        x2=self._remove_stopwords(x1)
        x3=self._remove_punctuation(x2)
        x4=self._remove_non_ascii(x3)
        x5=self._lower(x4)
        return x5
    
    def Summarize(self,url):
        import pandas as pd
        import datetime
        #import csv
        import numpy as np
        import time
        starttime=time.time()
        exclude=['twitter','glassdoor','youtube','wordpress','facebook','wikipedia','play.google','nasdaq']
        startTime_Main = datetime.datetime.now().strftime('%H:%M:%S')
        summarylist=[]
        summary1=""
        #print("---START TIME %s ---" % startTime_Main)
        for url in urls:
            if any (s in url for s in exclude):
                #print("Not allowed -",url)
                summary1="Not Allowed {}".format(url)
            else:
                try:
                    article=self.GetText(url)
                except BaseException as e:
                    article="ERROR {} {}".format(e,url)

                if not article.startswith("ERROR"):
                    print("fetching summary for - ",url)
                    summary1=self.Summary(article)
                else:
                    summary1=article

            summaryDict[url]=summary1
        #print("{:.2f}".format((time.time()-starttime)))
        return summary1
        

In [92]:
summarize=Extract()

In [93]:
summarize.GetText("https://www.theguardian.com/business/2018/jun/19/capita-awarded-mod-contract-despite-having-highest-risk-rating")

'First major deal completed by government since collapse of Carillion condemned by unions  Angela Monaghan   Tue 19 Jun 2018 20.24\xa0BST   Last modified on Wed 20 Jun 2018 00.05\xa0BST   Capita has been awarded a new contract from the Ministry of Defence to run UK military fire and rescue services despite a financial health assessment that attached the highest possible risk rating to the outsourcing company. The awarding of the contract was condemned by unions. It is the first major deal completed by the government since the collapse of the rival firm Carillion in January, which put the outsourcing of the public sector under the spotlight.  Capita scored 10 out of 10 for risk in a document prepared for the MoD. One indicates the lowest probability of distress and 10 the highest. The risk assessment document was updated on 7 June, according to the Financial Times. On Tuesday Capita confirmed it had been selected for the contract, which is estimated to be worth about £500m over 10 years

In [94]:
summarize.Summarize("https://www.theguardian.com/business/2018/jun/19/capita-awarded-mod-contract-despite-having-highest-risk-rating")

---START TIME 16:00:11 ---
fetching summary for -  https://www.theguardian.com/business/2018/jun/19/capita-awarded-mod-contract-despite-having-highest-risk-rating


'First major deal completed by government since collapse of Carillion condemned by unions  Angela Monaghan   Tue 19 Jun 2018 20.24\xa0BST   Last modified on Wed 20 Jun 2018 00.05\xa0BST   Capita has been awarded a new contract from the Ministry of Defence to run UK military fire and rescue services despite a financial health assessment that attached the highest possible risk rating to the outsourcing company. It is the first major deal completed by the government since the collapse of the rival firm Carillion in January, which put the outsourcing of the public sector under the spotlight. There needs to be an urgent investigation into how and why this contract was let to Capita.”  Capita came under the spotlight just days after the extent of Carillion’s woes came to light, when the new chief executive, Jon Lewis, announced a plans to tap the market for £700m of investment and suspended a dividend that was worth more than £200m to shareholders last year.'

In [ ]:
https://www.dailymail.co.uk/sciencetech/article-6144487/Scientists-release-highest-resolution-map-Antarctica-made.html

In [ ]:
df_flights.boxplot('dep_time','origin',rot = 30,figsize=(5,6))

In [172]:
data=pd.read_csv("./data/11SepALL.csv")

In [ ]:
data1=pd.read_csv("./data/ALL.csv")

In [174]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.article, data.type, test_size=0.33, random_state=42)

In [141]:
#pd.DataFrame({"Arcticles":X_test,"Category":y_test})

In [100]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(223, 15235)

In [101]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(223, 15235)

In [102]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(223, 15235)

In [158]:
newSum=ExtractNew()

In [ ]:
a=summarize.GetText(url)

In [125]:
url=["https://www.dailymail.co.uk/sport/football/article-6117823/Messi-Fabregas-Pique-happened-rest-Barcelonas-Class-87.html"
,"https://www.dailymail.co.uk/sciencetech/article-6144487/Scientists-release-highest-resolution-map-Antarctica-made.html"]

In [175]:

from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
text_clf = Pipeline([("pre",newSum),
                     ('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42,
                                           max_iter=5, tol=None)),
])


In [176]:
text_clf.fit(X=X_train,y=y_train)

Pipeline(memory=None,
     steps=[('pre', ExtractNew()), ('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_wo...ty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [184]:
predicted1 = text_clf.predict([" https://www.dailymail.co.uk/money/markets/article-6146323/Peppa-Pig-TV-firm-faces-SECOND-shareholder-revolt-bosss-62-pay-rise-just-3-years.html"])
#np.mean(predicted == y_test) 
print(predicted1[0])

money


In [179]:
np.mean(predicted == y_test) 

1.0

In [181]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))#,target_names=["Sports","Science","Femail"]))

             precision    recall  f1-score   support

     female       1.00      1.00      1.00        32
      money       1.00      1.00      1.00        27
sciencetech       1.00      1.00      1.00        54
      sport       1.00      1.00      1.00        33
  tvshowbiz       1.00      1.00      1.00         9

avg / total       1.00      1.00      1.00       155



In [185]:
metrics.confusion_matrix(y_test, predicted)

array([[32,  0,  0,  0,  0],
       [ 0, 27,  0,  0,  0],
       [ 0,  0, 54,  0,  0],
       [ 0,  0,  0, 33,  0],
       [ 0,  0,  0,  0,  9]], dtype=int64)

In [187]:
predicted

AttributeError: 'numpy.ndarray' object has no attribute 'plot'

In [182]:
X_test

55                                                                 https://www.dailymail.co.uk/femail/article-6148603/How-dress-like-grown-Sarah-Bailey.html
63                                                                https://www.dailymail.co.uk/femail/article-6150949/How-fans-finally-beat-ticket-touts.html
33                                       https://www.dailymail.co.uk/femail/article-6129433/Pioneering-software-reveals-Kate-Harry-Meghan-look-like-60s.html
297                              https://www.dailymail.co.uk/sciencetech/article-6140113/Galactic-wind-stops-early-galaxies-blowing-apart-observed-time.html
72                                                    https://www.dailymail.co.uk/femail/food/article-6142603/M-S-unveils-sneak-peek-festive-food-range.html
228                            https://www.dailymail.co.uk/sciencetech/article-6127529/AI-software-scans-satellite-images-predicts-residents-overweight.html
209                                              https://w